### **Load Environment variables from .env file**

In [2]:
import json
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
import openai
import tiktoken
from IPython.display import display, HTML, JSON, Markdown


In [3]:

load_dotenv()
# env variables that are used by LangChain
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_TYPE'] = "azure"
os.environ['OPENAI_API_VERSION'] = os.getenv("OPENAI_DEPLOYMENT_VERSION")
os.environ['OPENAI_API_BASE'] = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")

OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv(
    "OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

#gpt-4-turbo
OPENAI_GPT_4_TURBO_MODEL_NAME="gpt-4-turbo"
OPENAI_GPT_4_TURBO_DEPLOYMENT_NAME="gpt-4-turbo"

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_DEPLOYMENT_VERSION")
openai.api_base = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
openai.api_key = os.getenv("OPENAI_API_KEY")

### **Model initialization**

In [20]:

def init_llm(model=OPENAI_GPT_4_TURBO_MODEL_NAME,
             deployment_name=OPENAI_GPT_4_TURBO_DEPLOYMENT_NAME,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureChatOpenAI( deployment_name=deployment_name,
                      model=model,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      model_kwargs={"stop": ["<|im_end|>"],
                                    "top_p":top_p}
                      )
    return llm


### **Prompt engineering techniques**
Those techniques are not specific to summarization, but can be used for any task.

1. Use delimeters to clearly separate the exact text the model should summarize.

    Delimiters could be any kind of punctuation, that separates specific pieces of text. 
    Tripple quotes, Triple backtics, Triple dashes, Angle brackets, XML tags, etc.

2. Ask model for structured output, e.g json, html, etc. If you ask for a json define the structure of the json, e.g. what fields should be in the json.

3. Check assumptions required to do the task.

4. Few-shot prompting. Provide examples of completing tasks and then ask model to perform the task.



In [5]:
#Input text

text = f"""
**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Clerk**: Hello, John! My name is Sara, and I'm a customer service representative at Imaginal Bank. How can I assist you today?
**Customer**: Hi, Sara. I'm interested in your bank's investment programs. 
              Can you tell me more about them, especially in terms of risk management?

**Clerk**: Absolutely, John. We have a few key programs I can highlight.

First, there's our 'Balanced Growth Fund'. It's a diversified mutual fund that invests in a mix of equities and bonds to provide both growth and income, reducing risk through diversification. 

We also have the 'Index Tracker ETF', which is designed to replicate the performance of a specific market index. By spreading investments across the entire index, it inherently reduces the risk associated with individual stocks.

Additionally, for those with a lower risk tolerance, we have the 'Secure Income Bond Fund', which focuses on government and high-quality corporate bonds. 

Our financial advisors are always available to guide you in choosing the right program based on your financial goals and risk tolerance.

**Customer**: I see. Could you elaborate on how the Balanced Growth Fund manages risk?

**Clerk**: Sure. The Balanced Growth Fund mitigates risk by diversifying investments across a wide range of assets. If one investment performs poorly, it's likely to be offset by other investments that are performing well. Furthermore, our portfolio managers actively manage the fund, adjusting holdings based on changing market conditions to manage risk and enhance returns.

**Customer**: Does the bank provide any tools to monitor my investments?

**Clerk**: Yes, John. We offer an online platform called 'Imaginal Investor Dashboard'. It provides real-time tracking of your investments, balance updates, and market trends. You can also set up alerts to be notified about significant changes in your portfolio.

**Customer**: That sounds quite comprehensive. How can I get started?

**Clerk**: You can schedule an appointment with one of our financial advisors. They'll walk you through your options, help you understand your risk tolerance, and guide you in choosing the right investment program. Would you like me to arrange that for you?

**Customer**: Yes, please. That would be helpful.

**Clerk**: Fantastic, John! Let's get that set up for you...

"""


In [6]:
llm = init_llm()
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-turbo', temperature=0.0, model_kwargs={'stop': ['<|im_end|>'], 'top_p': 1}, openai_api_key='c50c3dd34a8c4fb8ba9bbc34d67ba9c1', openai_api_base='https://gpt35-instruct-demo.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=400, tiktoken_model_name=None, deployment_name='gpt-4-turbo', openai_api_type='azure', openai_api_version='2023-05-15')

In [7]:
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer in a clear and concise manner."},
            {"role": "user", "content": "Good morning, how are you today?"}]


answer = openai.ChatCompletion.create(engine="gpt-4-turbo",

                                      messages=messages,)
display(HTML("ChatCompletion (gpt-4-turbo) :" +
        answer.choices[0].message.content))

In [8]:
# Import LangChain schema classes for messages
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [9]:

messages = [
    SystemMessage(
        content="You're a nice and happy assistant. Answer users' questions happily.")
]
messages.append(
    HumanMessage(content="Good morning, how are you today?"))

answer = llm(messages)

display(HTML(f"gpt-4-turbo: {answer.content}"))


In [10]:


messages = [
    SystemMessage(
        content=f"""Summarize the text delimited by triple backticks into a summary of 20 words. 
        Text to summarize```{text}```
        Output the summary in the form: Summary: <summary>"""),
]

llm = init_llm("gpt-4-turbo", "gpt-4-turbo")
answer = llm(messages)
display(HTML(f"gpt-4-turbo: {answer.content}"))



In [11]:
#check llm object content. Pay attention that openai.api_resources.completion.Completion
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-turbo', temperature=0.0, model_kwargs={'stop': ['<|im_end|>'], 'top_p': 1}, openai_api_key='c50c3dd34a8c4fb8ba9bbc34d67ba9c1', openai_api_base='https://gpt35-instruct-demo.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=400, tiktoken_model_name=None, deployment_name='gpt-4-turbo', openai_api_type='azure', openai_api_version='2023-05-15')

In [12]:
# Check the specific conditions/assumptions in the input text

messages = [
    SystemMessage(
        content=f""" You will be provided with a text delimited by triple quotes. 
If it contains utterances of a Customer and a Clerk, summarize the text into single sentence.
If the text does not contain utterances of a Customer and a Clerk, 
then just write 'No customer and clerk utterances found'.
\"\"\" {text}  \"\"\"
Output a summary in the form: Summary: <summary> """),
]


llm = init_llm()
sum = llm(messages)
display(Markdown(sum.content))

Summary: John, a customer of Imaginal Bank, inquires about investment programs and their risk management, and Sara, the bank's clerk, explains the various funds available, the risk mitigation strategies, and the tools provided for investment monitoring, leading to John requesting an appointment with a financial advisor to get started.

#### Give the model time to think. Sub-tasks.

Very useful technique is to give the model time to think by specifying the strict steps (sub-tasks) to complete the task and asking for output in a specific format. This is a very powerful technique.


In [15]:
messages = [SystemMessage (content = f""" Your task is to perform the following actions:
1 - Summarize the text delimited by triple backticks into single sentence.
2 - Translate the summary into Portuguese.
3 - List the Customer questions.

Use the following output format:
Summary: <summary>
Translated summary: <portuguese summary>
Customer questions: <enumerated customer questions>

Text: ```{text}``` 
""") ]

llm = init_llm()
sum = llm(messages)
display(Markdown(sum.content))

Summary: John, a customer of Imaginal Bank, inquires about the bank's investment programs and risk management, and Sara, the customer service representative, explains the various investment options, risk mitigation strategies, and tools for monitoring investments, offering to set up an appointment with a financial advisor to get started.

Translated summary: John, um cliente do Imaginal Bank, pergunta sobre os programas de investimento do banco e gestão de riscos, e Sara, a representante de atendimento ao cliente, explica as várias opções de investimento, estratégias de mitigação de riscos e ferramentas para monitorar investimentos, oferecendo-se para marcar uma consulta com um consultor financeiro para começar.

Customer questions:
1. Can you tell me more about the bank's investment programs, especially in terms of risk management?
2. Could you elaborate on how the Balanced Growth Fund manages risk?
3. Does the bank provide any tools to monitor my investments?
4. How can I get started?

##### Let's do some math

In [16]:
messages = [SystemMessage(content =  f""" Determine if the student's solution delimited by the triple backticks is correct or not.
Question: When I was 2 years old, my sister was twice my age. I'm now 40 years old. How old is my sister now? 
Student's answer: ```The sister is now 80 years old.``` 
If the student answer is correct, write 'Correct', otherwise write 'Incorrect'. Explain your answer.  
<|im_end|>
""")]

llm = init_llm()
sum = llm(messages)
display(HTML(sum.content))

##### Ask model to do its own solution and then to compare both solutions and conclude which is correct.
 Define the task as a list of sub-tasks and ask the model to perform them in a specific order (splitting the task into sub-tasks technique). 
 Then ask the model to compare its own solution with the solution provided by the model and conclude which is correct.
 Ask model to share it reasoning for the conclusion.

In [17]:
Question = f"""When I was 2 years old my sister was twice of my age . I'm now 40 years old, how old is my sister now?"""
Student_Solution = f""" The sister is now 80 years old."""

messages = [ SystemMessage (content =  f""" Determine if the Student's Solution for the Question is correct or not.
To solve the problem do the following:
1 - First, work out your OWN solution to the problem. Evaluate your final result to make sure it is correct and adheres to the question's conditions. 
    Reason about every step of your solution and make sure it is correct. 
2 - Second, compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Student's Solution: ```{Student_Solution}```
Question: ```{Question}```


Use the following output format:
Actual solution steps: <your own solution steps>
\nStudent's solution: <student's solution>
\nStudent's solution is correct: <true/false>

""")]

llm = init_llm()
sum = llm(messages)
display(Markdown(sum.content))  

Actual solution steps:
1. When the person was 2 years old, their sister was twice their age, which means the sister was 2 * 2 = 4 years old at that time.
2. To find out the age difference between the person and their sister, we subtract the person's age at that time from the sister's age at that time: 4 - 2 = 2 years.
3. The age difference between the person and their sister will always be the same. So, if the person is now 40 years old, we add the age difference to find the sister's current age: 40 + 2 = 42 years old.

The sister is now 42 years old.

Student's solution: The sister is now 80 years old.

Student's solution is correct: False

#### Model Hallucinations and how to avoid them

Hallucination is when the model generates text that is not supported by the input.
To reduce hallucinations, use the techniques listed above, and strictly instruct the model to find the relevant information in the input text. 
If the information is not in the input, instruct the model to generate a specific output, e.g. "I don't know the answer to this question". 

#### Iterative Prompt Development
Prompt engineering is an iterative process. You almost never get the prompt right the first time.
You start with the idea, then you implement the prompt, get the experimental resuslts, do the Error Analysis and iterate again.

**Try -> Analyze -> Clarify Instructions -> Try again**

In more advanced phases refine prompts with a batch of examples.

### Summarization 

In [18]:
llm = init_llm()
prompt_prefix = [SystemMessage(content = f""" Summarize the text delimited by triple backticks into 2-3 sentences: ```{text}``` """)]
text=" "
with open("./data/bank-call-center-transcript.txt", 'r') as file:
        text = file.read()
        
sum = llm(prompt_prefix)
display(Markdown(sum.content))

John, a customer of Imaginal Bank, inquires about the bank's investment programs and their risk management strategies. Sara, the customer service representative, explains that the bank offers various investment options such as the Balanced Growth Fund, Index Tracker ETF, and Secure Income Bond Fund, each with different levels of risk management. She also mentions that the bank provides an online platform for investment monitoring and offers personalized guidance from financial advisors to help customers choose the right investment program based on their risk tolerance.

#### More advanced prompts

In [19]:
llm = init_llm()

text=" "
with open("./data/bank-call-center-transcript.txt", 'r') as file:
        text = file.read()
        
prompt_prefix = [SystemMessage ( content = f""" Prepare a summary for the text delimited by the triple backticks```{text}``` based on the points mentioned below. 
Please evaluate whether the clerk successfully accomplished the following tasks:

Greeting the customer politely and professionally.
Accurately understanding the customer's inquiry.
Providing clear and detailed information in response.
Asking questions as needed for clarification.
Discussing both benefits and risks with the customer.
Explaining the tools and resources available to the customer.
Inviting the customer to take further action.
Offering assistance for the next steps.
Ending the conversation on a positive note.
""")
]

sum = llm(prompt_prefix)
display(Markdown(sum.content))

The clerk, Sara, greeted the customer, John, politely and professionally, introducing herself and asking how she could assist him. She accurately understood John's inquiry about the bank's investment programs and their risk management strategies. Sara provided clear and detailed information about the 'Balanced Growth Fund', 'Index Tracker ETF', and 'Secure Income Bond Fund', explaining how each manages risk and suits different risk tolerances.

Although Sara did not ask questions for clarification, the conversation did not seem to require it, as John's questions were straightforward. She discussed the benefits of each investment option and touched on risk management, particularly for the Balanced Growth Fund, explaining diversification and active management by portfolio managers.

Sara informed John about the 'Imaginal Investor Dashboard', a tool for monitoring investments, and explained its features. She invited John to take further action by offering to schedule an appointment with a financial advisor to guide him through the investment process. Sara offered assistance for the next steps and ended the conversation on a positive note by expressing eagerness to set up the appointment for John. Overall, the clerk successfully accomplished the tasks listed.

##### Output in tabular format

In [21]:
llm = init_llm()

text=" "
with open("./data/bank-call-center-transcript.txt", 'r') as file:
        text = file.read()
        
prompt_prefix = [ SystemMessage( content = f"""Prepare a summary for the text delimited by the triple backticks:```{text}``` based on the points mentioned below.
Generate the output as a HTML table, with 2 columns: Customer and Clerk.

Assign a color code to each item based on the clerk's performance - 
items that were successfully addressed should be marked in green, 
whereas items that were not met should be highlighted in red.

Here are the evaluation criterias to consider:

Did the clerk greet the customer in a polite and professional manner?
Did the clerk begin the conversation on a negative note?
Did the clerk understand the customer's inquiry?
Did the clerk provide clear and detailed information?
Did the clerk ask questions to clarify the situation?
""" )]

res = llm(prompt_prefix)
                     
display(HTML(res.content))

Did the clerk greet the customer in a polite and professional manner?,"Yes, the clerk greeted the customer politely with ""Hello, John!"" and introduced herself as a customer service representative ready to assist."
Did the clerk begin the conversation on a negative note?,"No, the clerk did not begin the conversation on a negative note."
Did the clerk understand the customer's inquiry?,"Yes, the clerk understood the customer's interest in investment programs and risk management and provided relevant information."
Did the clerk provide clear and detailed information?,"Yes, the clerk provided clear and detailed information about the investment programs, including the Balanced Growth Fund, Index Tracker ETF, and Secure Income Bond Fund, as well as the risk management strategies and tools for monitoring investments."
Did the clerk ask questions to clarify the situation?,"No, the clerk did not ask questions to further clarify the customer's situation or needs."


##### Output as JSON

In [25]:
llm = init_llm()

text=" "
with open("./data/bank-call-center-transcript.txt", 'r') as file:
        text = file.read()
        
prompt_prefix = [ SystemMessage ( content = f"""" Evaluate the clerk performance from the text delimited by the triple backticks: ```{text}``` based on the points mentioned below.

Did the clerk greet the customer in a polite and professional manner?
Did the clerk comprehend the customer's inquiry accurately?
Did the clerk provide comprehensive and clear information?
Did the clerk ask relevant questions to clarify the customer's situation?
Did the clerk explain the benefits and potential risks to the customer?
Did the clerk detail the tools and resources available to the customer?
Did the clerk encourage the customer to take further action?
Did the clerk offer assistance with proceeding to the next steps?
Did the clerk end the interaction on a positive and upbeat note?"

The output should be presented in JSON format, adhering to the following key-value pairs:

"Greet the Customer Politely and Professionally": "Yes/No",
"Understand the Customer's Inquiry": "Yes/No",
"Provide Clear and Detailed Information": "Yes/No",
"Ask Clarifying Questions": "Yes/No",
"Discuss the Benefits and Risks": "Yes/No",
"Explain Available Tools and Resources": "Yes/No",
"Invite Further Action": "Yes/No",
"Offer to Assist with Next Steps": "Yes/No",
"End on a Positive Note": "Yes/No"

""")]

res = llm( prompt_prefix )
res.content



'```json\n{\n  "Greet the Customer Politely and Professionally": "Yes",\n  "Understand the Customer\'s Inquiry": "Yes",\n  "Provide Clear and Detailed Information": "Yes",\n  "Ask Clarifying Questions": "No",\n  "Discuss the Benefits and Risks": "Yes",\n  "Explain Available Tools and Resources": "Yes",\n  "Invite Further Action": "Yes",\n  "Offer to Assist with Next Steps": "Yes",\n  "End on a Positive Note": "Yes"\n}\n```'

### Summarize large documents
To summarize larger documents, we can split the document into smaller chunks and summarize each chunk separately. We can then combine the summaries of each chunk to get the final summary.
LangChain has a built-in chain for doing that. 



In [26]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

from langchain.document_loaders import PyPDFLoader
llm = init_llm()

large_pdf_path = "./data/Large_language_model.pdf"
loader = PyPDFLoader(large_pdf_path)

# output type is List[Document]
pages = loader.load_and_split()

# count tokens in the document
total_tokens = 0
encoder = tiktoken.get_encoding("cl100k_base")

for page in pages:
    total_tokens += len(encoder.encode(page.page_content))

# The document has more 13000 tokens. This is too many for the LLM to process in one go.
# This is whay we split the document into smaller chunks.
print(f"Total tokens in the document: {total_tokens}")

Total tokens in the document: 13420


In [27]:
from langchain.chains.summarize import load_summarize_chain

# create summarization chain
# map_reduce summarizes each document on it's own in a "map" step and then "reduce" the summaries into a final summary
summary_chain = load_summarize_chain( 
    llm=llm, chain_type='map_reduce', verbose=False)

##### There are following chain_types: stuff, map_reduce, refine, map-rerank
See here for the details: https://docs.langchain.com/docs/components/chains/index_related_chains

In [30]:
# Many calls in loop causing RateLimitError: Too Many Requests. In this sum = summary_chain.run(pages[0:10]) 
sum = summary_chain.run(pages)

display(Markdown(sum))

KeyboardInterrupt: 

In [15]:
#TODO - Upgrade LangChain version to the latest
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain, StuffDocumentsChain

llm = init_llm()

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:
<|im_end|>"""

map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. 
Summary:

<|im_end|>
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=2000,
)

In [16]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


In [47]:
map_reduce_chain.run(pages)

'Large language models have become powerful tools in natural language processing, with their capabilities and performance constantly improving. However, concerns about training costs and potential biases remain. Pre-training and fine-tuning techniques are used to optimize these models for specific tasks. Open-source models and collaborations have made these models more accessible, leading to advancements in dialogue and conversational AI. Ethical considerations and responsible AI are also important factors in their development and use. The release of new models, partnerships, and advancements in training data and computing are driving the future of AI and language models, with potential for impact in various industries.'